In [ ]:
!git clone https://github.com/YasasviPeruvemba/deblur-gan.git

In [ ]:
%%bash
mkdir /content/deblur-gan/gopro/train -p
mkdir /content/deblur-gan/logs
# Find Dataset from here - https://drive.google.com/file/d/1CPMBmRj-jBDO2ax4CxkBs9iczIFrs8VA/view?usp=sharing
gdown --id "1CPMBmRj-jBDO2ax4CxkBs9iczIFrs8VA" 
mv blurred_sharp.zip /content/deblur-gan/gopro/train/
cd /content/deblur-gan/gopro/train
unzip blurred_sharp.zip -q
mv "./blurred_sharp/blurred" "./blurred_sharp/blur"
rm -r /content/deblur-gan/gopro/train/__MACOSX
rm blurred_sharp.zip
cd /content/deblur-gan

In [ ]:
%cd /content/deblur-gan/
%pip install -r requirements/requirements.txt
%pip install -e .

In [6]:
!python scripts/organize_gopro_dataset.py --dir_in=./gopro --dir_out=./images

dir:   0% 0/1 [00:00<?, ?it/s]
image_folders:   0% 0/1 [00:00<?, ?it/s]
image_folders: 100% 1/1 [00:03<00:00,  3.88s/it]
dir: 100% 1/1 [00:03<00:00,  3.88s/it]


In [ ]:
# CHECK IF THE PREPROCESSING WORKED
import matplotlib.pyplot as plt
a = plt.imread("./images/train/A/blurred_sharp_257.png")
plt.imshow(a);

In [ ]:
!python scripts/deblur_image.py --weight_path=/content/deblur-gan/generator_195_204.h5 --input_dir=./images/ --output_dir=./results/

In [ ]:
# !rm -rf custom_test/input/.ipynb_checkpoints/* custom_test/output/.ipynb_checkpoints/* custom_test/.ipynb_checkpoints/*
# !rmdir custom_test/input/.ipynb_checkpoints custom_test/output/.ipynb_checkpoints custom_test/.ipynb_checkpoints

# Gradients with respect to images

In [20]:
import os
import datetime
import click
import numpy as np
import tqdm
import keras.backend as K
import tensorflow as tf

from deblurgan.utils import load_images, write_log
from deblurgan.losses import wasserstein_loss, perceptual_loss_100
from deblurgan.model import generator_model, discriminator_model, generator_containing_discriminator_multiple_outputs

import numpy as np

In [ ]:
def get_gen_grads(batch_size=2):
    data = load_images('./images/train', batch_size)
    y_train, x_train = data['B'], data['A']

    g = generator_model()
    g.load_weights('generator_195_204.h5')
    d = discriminator_model()
    d.load_weights('discriminator_195.h5')

    model = generator_containing_discriminator_multiple_outputs(g, d)

    sess = K.get_session()
    perceptual_loss_func = tf.gradients(perceptual_loss_100(tf.convert_to_tensor(y_train), model.output[0]), model.input)
    wasserstein_loss_func = tf.gradients(wasserstein_loss(tf.convert_to_tensor(np.ones(y_train.shape[0], dtype=np.float32)), model.output[1]), model.input)
    gen_gradients = sess.run([perceptual_loss_func, wasserstein_loss_func], feed_dict={model.input : x_train})

    print(x_train.shape)
    print(gen_gradients[0][0].shape)
    print(gen_gradients[1][0].shape)

# Send batch_size small as it is computationally heavy
get_gen_grads(batch_size=2)


In [ ]:
def get_discrimator_grads(batch_size=2):
    data = load_images('./images/train', batch_size)
    y_train, x_train = data['B'], data['A']

    model = discriminator_model()
    model.load_weights('discriminator_195.h5')

    ground_truths_real = tf.convert_to_tensor(np.ones((batch_size, 1), dtype=np.float32))
    ground_truths_fake = tf.convert_to_tensor(np.zeros((batch_size, 1), dtype=np.float32))

    ground_truth = tf.concat([ground_truths_real, ground_truths_fake], axis=0)[0]
    input = np.append(y_train, x_train, axis=0)

    sess = K.get_session()
    grad_func = tf.gradients(wasserstein_loss(ground_truth, model.output), model.input)
    disc_gradients = sess.run(grad_func, feed_dict={model.input : input})[0]

    print(input.shape)
    print(disc_gradients.shape)
    print(disc_gradients)

# Send batch_size small as it is computationally heavy
get_discrimator_grads(batch_size=2)